In [1]:
import numpy as np
from skimage import io, util, measure
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd

from os import path
import stackview
import matplotlib.pyplot as plt


# 0. Load datasets

In [23]:
# Load segmentations
dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/09-29-2025 Ear palbo pilot/M1tail 3017 H2B FUCCI/Left ear/1. Day 1/R1'

masks = io.imread(path.join(dirname,'G_reg_decon_cp_masks.tif'))
heightmap = io.imread(path.join(dirname,'heightmap.tif'))
height_image = io.imread(path.join(dirname,'height_image.tif'))

In [24]:
df = pd.DataFrame(measure.regionprops_table(masks,properties={'label','area','centroid','bbox'}))

# Look at each XY coord and look up heightmap
Z = heightmap[ df['centroid-1'].round().astype(int), df['centroid-1'].round().astype(int) ]
df['Corrected Z'] = df['bbox-0'] - Z


In [25]:
stackview.switch([masks,height_image],
                 colormap=['pure_magenta','pure_green'],
                zoom_factor=0.5,
                toggleable=True)

In [26]:
df.to_excel(path.join(dirname,'pre_gated.xlsx'))

cluster_plot = stackview.clusterplot(image=masks,
                     labels=masks,
                     df=df,
                     column_x="area",
                     column_y="Corrected Z", 
                     zoom_factor=0.5,
                     alpha=0.5)
cluster_plot

In [40]:
basals = df[df['selection']]
basal_masks = masks.flatten()
I = np.in1d(basal_masks, basals.index)
basal_masks[~I] = 0
basal_masks = basal_masks.reshape(masks.shape)

/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_93781/2864155300.py:3: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  I = np.in1d(basal_masks, basals.index)


In [41]:
stackview.switch({'all_masks':masks,
                  'basals':basal_masks},
                 colormap=['pure_magenta','pure_green'],
                 toggleable=True,
                 zoom_factor=0.5)

In [43]:
io.imsave(path.join(dirname,'G_reg_decon_cp_masks_basals.tif'),basal_masks)

/opt/miniconda3/envs/jupyter/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/09-29-2025 Ear palbo pilot/M1tail 3017 H2B FUCCI/Left ear/1. Day 1/R1/G_reg_decon_cp_masks_basals.tif is a low contrast image
  return func(*args, **kwargs)
